In [467]:
import pandas as pd
import re
import source.utils.associate as ass

from source.utils import am_notebooks as an
from source.utils.general import confirm_dir, snake_to_camel

SAMPLE_COLS = ['f', 'f1', 'f2', 'N', 
               'exp_f', 'unexp_f', 'unexp_r',
               'P1',  'P2',
               'dP1', 'dP2',
               'deltaP_mean',
            #    'deltaP_max',
               'LRC', 'G2',
               ]
MIN = 500
POLAR_ADV_MIN = 1000
POLAR_ADJ_MIN = 300
POLAR_BIG_MIN = 100

MIR_BLIND_COLOR = 'BuPu'
SUP_BLIND_COLOR = 'RdPu'
MIR_POLAR_COLOR = 'purple_teal'
SUP_POLAR_COLOR = 'anastasia'

VERBOSE = True
TEX_AM_SAMPLE_DIR = an.LATEX_TABLES.joinpath('ch4/samples')
confirm_dir(TEX_AM_SAMPLE_DIR)

def get_polar_label(unit):
    long_form = {'adv':'adverb', 'adj': 'adjective', 'bigram': 'bigram'}
    return f'{long_form[unit.lower()]} Polarity'.title()
print(get_polar_label('adv'))

Adverb Polarity


In [468]:

def _load_amdf(dataset='direct', sampling='ALL', f_floor=MIN, polar=False, unit='adv'):
    #^ e.g. polar/RBdirect/adv/extra/polarized-adv_NEQ-direct_min500x_extra.parq
    if polar:
        path_stub = ('polar/'
                     + (f'RB' if dataset == 'direct' else '')
                     + f'{dataset}/{unit}/extra/polarized-{unit}'
                     + f'_{sampling}-{dataset}_min{f_floor}x_extra.parq')
    else:
        path_stub = ('adv_adj/'
                     + f'ANY{dataset}/extra/AdvAdj_{sampling}'
                     + f'_any-{dataset}_final-freq_min{f_floor}x_extra.parq')
    _df = ass.adjust_am_names(
        pd.read_parquet(
            ass.AM_DF_DIR.joinpath(path_stub))
    )
    if not polar:
        _df = _df.assign(key=_df.l1.astype('string') + '~' +
                     _df.l2.astype('string')).set_index('key')
    _df = _df.filter(SAMPLE_COLS).sort_values(['LRC','unexp_r']).rename(columns={'deltaP_mean':'dPavg'})
    _df.columns = _df.columns.to_series().apply(snake_to_camel)
    _df['LogR'] =  ass.am.log_ratio(ass.fq.expected_frequencies(_df, observed=True))
    return _df

# tester = _load_amdf(f_floor=2000, polar=True)
# tester

In [504]:
def latexify_sample(sample_df: pd.DataFrame,
                    cmap: str = None,
                    mirror: bool = False,
                    neq: bool = False,
                    comparison_label: str = 'context--blind',
                    dry: bool = False,  # ! Don't write to file!,
                    f_floor: int = MIN
                    ):
    caption, label = _set_meta_strings(mirror, neq, comparison_label)
    tex_tab_str = _build_tex_table(sample_df, cmap, caption, label)
    tex_tab_str = '\n'.join(
        (
            f'%> Frequency Floor of Sample = {f_floor:,}',
            f'%> Caption: {caption}',
            f'%> Label: {label}',
            '\singlespacing\\footnotesize',
            tex_tab_str,
            '\\normalspacing\\normalsize'
        ))
    am_sample_path = TEX_AM_SAMPLE_DIR.joinpath(f'{label}.tex')
    print(f'Writing "{caption}" Table to\n  {am_sample_path}')
    if not dry:
        am_sample_path.write_text(tex_tab_str, encoding='utf-8')
    else:
        print('\n<< Dry Run --- Nothing Saved >>')
        print(f'Frequency Floor of Sample = {f_floor:,}')
        print(tex_tab_str)


def _set_meta_strings(mirror, neq, comparison_label):
    sampling_tag = 'NEQ' if neq else 'ALL'
    dataset_tag = ('mirror subset' if mirror else 'superset')
    caption = ('\\cmbr{'
               + sampling_tag
               + '+} '
               + dataset_tag.title()
               + f': {comparison_label.title()} AM Sample')
    print(caption)
    label = '-'.join([f'AM={comparison_label.split("-")[-1].lower()}',
                      dataset_tag.split(' ')[0].replace('set', ''),
                     sampling_tag]).strip('-').replace(' ', '-')
    print(label)
    return caption, label


def _build_tex_table(sample_df, cmap, caption, label):
    max_digits = len(str(sample_df.loc["N", :].max()).split('.')[0])
    print(f'N = {int(sample_df.T.N.max()):,}')
    print(f'max digits = {max_digits}')

    sample_df = _adjust_header(sample_df)
    sample_for_sty = sample_df.convert_dtypes()
        
    sample_sty = an.format_zeros(
        sample_for_sty
        .style.background_gradient(cmap, axis=1)
        .format(escape='latex')
        .format_index(escape='latex')
        # .format_index(escape='latex', axis=1)
    )
    column_format = _set_col_format(n_cols=len(sample_df.columns),
                                    max_digits=max_digits,
                                    blind='blind' in label)
    _tex_table_str = sample_sty.to_latex(
        siunitx=True,
        multicol_align='c',
        environment='longtable',
        caption=caption,
        label='tab:'+label.replace('=', '-'),
        convert_css=True,
        sparse_columns=True,
        column_format=column_format,
        hrules='\\midrule')
    # print(re.findall(r'\\\s*\n([\w]+)\s*(?=&)', _tex_table_str))
    _tex_table_str = re.sub(
        r'(?<=\n)([\w]+)\s*(?=&)', '__cmtt{'+r'\1'+'} ', _tex_table_str)
    return _tex_table_str.replace('__cmtt', '\\cmtt')

def _adjust_header(sample_df):
    t_sample_df = sample_df.T
    t_sample_df['key'] = t_sample_df.index.str.replace('~', '$\\sim$')
    if sample_df.columns.str.contains('[A-Z_]').any():
        t_sample_df = an.assign_polarity(t_sample_df)
        t_sample_df['polarity'] = t_sample_df.polarity.map(
            {'neg': '$(-)$', 'pos': '$(+)$'})
        t_sample_df = t_sample_df.set_index(['polarity', 'key'])
    else:
        t_sample_df = t_sample_df.set_index(['key'])
    return t_sample_df.sort_index().T


def _set_col_format(n_cols: int, max_digits: int, blind: bool):
    siparam = [f'table-format=-{max_digits}.3', 'drop-zero-decimal']
    if blind:
        siparam.append('negative-color=BrickRed')
    else: 
        siparam.append('round-minimum=0.001')
    column_format = ('r*{'
                     + f'{n_cols}'
                     + '}{S'+repr(siparam).replace("'", "")
                     + '}')

    return column_format


latexify_sample(_load_amdf(f_floor=2000, polar=False).sample(2).T,
                neq=True, mirror=True, f_floor=2000, dry=True)

\cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample
AM=blind-mirror-NEQ
N = 71,961,373
max digits = 8
Writing "\cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=blind-mirror-NEQ.tex

<< Dry Run --- Nothing Saved >>
Frequency Floor of Sample = 2,000
%> Frequency Floor of Sample = 2,000
%> Caption: \cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample
%> Label: AM=blind-mirror-NEQ
\singlespacing\footnotesize
\begin{longtable}{r*{2}{S[table-format=-8.3, drop-zero-decimal, negative-color=BrickRed]}}
\caption{\cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample} \label{tab:AM-blind-mirror-NEQ} \\
\toprule
{key} & {all$\sim$happy} & {easily$\sim$available} \\
\midrule
\endfirsthead
\caption[]{\cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample} \\
\toprule
{key} & {all$\sim$happy} & {easily$\sim$available} \\
\midrule
\endhead
\midrule
\multicolumn{3}{r}{Continued on next page} \\
\midrule
\endfoot
\bot

## Context-Blind

In [240]:
all_super_blind1k = _load_amdf(f_floor=1000)
top_lrc_min1k = ass.adjust_am_names(all_super_blind1k).nlargest(20, ['LRC'])
(pd.concat([top_lrc_min1k.nlargest(1, 'LRC'),
            ass.adjust_am_names(all_super_blind1k).nlargest(1, 'dPavg'),
            top_lrc_min1k.nsmallest(1, 'dP2'),
            top_lrc_min1k.nsmallest(1, 'dP1')]).select_dtypes(include='number').drop_duplicates().T.convert_dtypes().style
 ).background_gradient(axis=1)

key,thinly~veiled,scantily~clad,far~away,upwardly~mobile
f,1333.000000,4638.000000,1150.000000,1893.000000
f1,1746.000000,4682.000000,402768.000000,2002.000000
f2,1705.000000,6562.000000,1157.000000,20114.000000
N,71961373.000000,71961373.000000,71961373.000000,71961373.000000
expF,0.041368,0.426941,6.475732,0.559581
unexpF,1332.958632,4637.573059,1143.524268,1892.440419
unexpR,0.999969,0.999908,0.994369,0.999704
P1,0.781818,0.706797,0.993950,0.094114
P2,0.763459,0.990602,0.002855,0.945554
dP1,0.781812,0.706796,0.988369,0.094112


In [241]:
top_lrc_min1k = (all_super_blind1k).nlargest(20, ['LRC'])
print(pd.concat([top_lrc_min1k.nlargest(1, 'LRC'), 
            (all_super_blind1k).nlargest(1, 'dPavg'), 
            top_lrc_min1k.nsmallest(1, 'dP2'), 
            top_lrc_min1k.nsmallest(1, 'dP1')])
 .select_dtypes(include='number').T.convert_dtypes()
 .style.background_gradient(axis=1)
 .format(escape='latex').format_index(escape='latex').format_index(escape='latex', axis=1)
 .to_latex(siunitx=True, environment='longtable', column_format='S[table-format=8.2]', convert_css=True)
)

\begin{longtable}{S[table-format=8.2]}
{key} & {thinly\textasciitilde veiled} & {scantily\textasciitilde clad} & {far\textasciitilde away} & {upwardly\textasciitilde mobile} \\
\endfirsthead
{key} & {thinly\textasciitilde veiled} & {scantily\textasciitilde clad} & {far\textasciitilde away} & {upwardly\textasciitilde mobile} \\
\endhead
\multicolumn{5}{r}{Continued on next page} \\
\endfoot
\endlastfoot
f & {\cellcolor[HTML]{F7F0F7}} \color[HTML]{000000} 1333.000000 & {\cellcolor[HTML]{023858}} \color[HTML]{F1F1F1} 4638.000000 & {\cellcolor[HTML]{FFF7FB}} \color[HTML]{000000} 1150.000000 & {\cellcolor[HTML]{D9D8EA}} \color[HTML]{000000} 1893.000000 \\
f1 & {\cellcolor[HTML]{FFF7FB}} \color[HTML]{000000} 1746.000000 & {\cellcolor[HTML]{FEF6FB}} \color[HTML]{000000} 4682.000000 & {\cellcolor[HTML]{023858}} \color[HTML]{F1F1F1} 402768.000000 & {\cellcolor[HTML]{FFF7FB}} \color[HTML]{000000} 2002.000000 \\
f2 & {\cellcolor[HTML]{FBF3F9}} \color[HTML]{000000} 1705.000000 & {\cellcolor[HTML]{

In [244]:
def _pull_blind_samples(all_amdf, neq_amdf, k=4):
    seeking_sample = True
    while seeking_sample:
        sample_NEQ = neq_amdf.sample(k).T
        if all(sample_NEQ.columns.to_series().str.extract(r'^(?P<l1>.+)~(?P<l2>.+)$').nunique() == k): 
            seeking_sample = False
    sample_NEQ = sample_NEQ.convert_dtypes()
    sample_ALL = all_amdf.filter(sample_NEQ.columns, axis=0).T.convert_dtypes()
    if VERBOSE:
        an.nb_display(sample_NEQ.style.background_gradient(axis=1))
        an.nb_display(sample_ALL.style.background_gradient(axis=1))
    else:
        an.nb_display(sample_NEQ.columns.to_series().to_frame('sample'))
    return {'ALL': sample_ALL, 'NEQ':sample_NEQ}

# _pull_blind_samples(_load_amdf(f_floor=2000), _load_amdf(f_floor=2000, sampling='NEQ'))

### Collect *<font color=DarkViolet>Superset</font>* <font color=Teal>Context-Blind</font> AM Samples and Save as $\LaTeX$ Tables

In [138]:

# > load AM dataframes
all_super_blind = _load_amdf()
neq_super_blind = _load_amdf(sampling='NEQ')
an.nb_display(pd.concat([all_super_blind.head(2), neq_super_blind.head(2)]))

,f,f1,f2,N,expF,unexpF,unexpR,P1,P2,dP1,dP2,dPavg,LRC,G2,LogR
key,,,,,,,,,,,,,,,
so~likely,699,"5,173,538","884,149","71,961,373","63,564.36","-62,865.36",-89.94,0.00,0.00,-0.07,-0.01,-0.04,-6.31,"-124,953.4",-6.61
most~different,"1,284","7,120,428","802,192","71,961,373","79,375.23","-78,091.23",-60.82,0.00,0.00,-0.10,-0.01,-0.06,-5.88,"-154,697.9",-6.10
more~good,800,"553,482","202,048","6,347,362","17,618.33","-16,818.33",-21.02,0.00,0.00,-0.09,-0.03,-0.06,-4.32,"-30,751.3",-4.59
as~sure,829,"656,683","134,058","6,347,362","13,869.32","-13,040.32",-15.73,0.01,0.00,-0.10,-0.02,-0.06,-3.95,"-23,069.3",-4.21


In [189]:

# > Pull samples
blind_super_samples = _pull_blind_samples(all_amdf=all_super_blind, neq_amdf=neq_super_blind)

key,yet~final,really~happy,physically~able,so~clear
f,640.00,"3,000.00",522.00,"3,583.00"
f1,"53,779.00","182,968.00","7,930.00","566,272.00"
f2,"1,212.00","47,131.00","22,885.00","83,958.00"
N,"6,347,362.00","6,347,362.00","6,347,362.00","6,347,362.00"
expF,10.27,"1,358.59",28.59,"7,490.21"
unexpF,629.73,"1,641.41",493.41,"-3,907.21"
unexpR,0.98,0.55,0.95,-1.09
P1,0.53,0.06,0.02,0.04
P2,0.01,0.02,0.07,0.01
dP1,0.52,0.04,0.02,-0.05


key,yet~final,really~happy,physically~able,so~clear
f,640.00,"35,529.00","1,955.00","10,704.00"
f1,"94,755.00","1,945,388.00","63,922.00","5,173,538.00"
f2,"5,736.00","420,680.00","210,786.00","346,404.00"
N,"71,961,373.00","71,961,373.00","71,961,373.00","71,961,373.00"
expF,7.55,"11,372.57",187.24,"24,904.11"
unexpF,632.45,"24,156.43","1,767.76","-14,200.11"
unexpR,0.99,0.68,0.90,-1.33
P1,0.11,0.08,0.01,0.03
P2,0.01,0.02,0.03,0.00
dP1,0.11,0.06,0.01,-0.04


### Collect *<font color=DeepSalmon>Mirror Subset</font>* <font color=Teal>Context-Blind</font> AM Samples and Save as $\LaTeX$ Tables

In [246]:
# > load AM dataframes
all_mirror_blind = _load_amdf(dataset='mirror', f_floor=300)
neq_mirror_blind = _load_amdf(dataset='mirror', f_floor=300, sampling='NEQ')


In [269]:

# > Pull samples
blind_mirror_samples = _pull_blind_samples(all_amdf=all_mirror_blind, neq_amdf=neq_mirror_blind)

key,more~enjoyable,particularly~good,that~simple,so~beautiful
f,488.00,416.00,487.00,464.00
f1,"121,848.00","10,020.00","4,538.00","47,033.00"
f2,827.00,"13,484.00","7,580.00","2,894.00"
N,"583,470.00","583,470.00","583,470.00","583,470.00"
expF,172.71,231.56,58.95,233.28
unexpF,315.29,184.44,428.05,230.72
unexpR,0.65,0.44,0.88,0.50
P1,0.59,0.03,0.06,0.16
P2,0.00,0.04,0.11,0.01
dP1,0.38,0.01,0.06,0.08


key,more~enjoyable,particularly~good,that~simple,so~beautiful
f,"1,241.00",475.00,529.00,"1,660.00"
f1,"285,773.00","12,946.00","5,465.00","129,421.00"
f2,"2,315.00","31,461.00","25,344.00","7,912.00"
N,"1,680,633.00","1,680,633.00","1,680,633.00","1,680,633.00"
expF,393.64,242.35,82.41,609.28
unexpF,847.36,232.65,446.59,"1,050.72"
unexpR,0.68,0.49,0.84,0.63
P1,0.54,0.02,0.02,0.21
P2,0.00,0.04,0.10,0.01
dP1,0.37,0.01,0.02,0.13


### 💾 Save <font color=Teal>Context-Blind</font>  Samples as $\LaTeX$ tables

In [505]:
# > Save Samples as $\LaTeX$ tables
for sam in blind_super_samples:

    latexify_sample(blind_super_samples[sam],cmap=SUP_BLIND_COLOR, neq=(sam=='NEQ'))

for sam in blind_mirror_samples:
    latexify_sample(blind_mirror_samples[sam], 
                    mirror=True, 
                    cmap=MIR_BLIND_COLOR,
                    f_floor=300, 
                    neq=(sam=='NEQ'))


\cmbr{ALL+} Superset: Context--Blind AM Sample
AM=blind-super-ALL
N = 71,961,373
max digits = 8
Writing "\cmbr{ALL+} Superset: Context--Blind AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=blind-super-ALL.tex
\cmbr{NEQ+} Superset: Context--Blind AM Sample
AM=blind-super-NEQ
N = 6,347,362
max digits = 7
Writing "\cmbr{NEQ+} Superset: Context--Blind AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=blind-super-NEQ.tex
\cmbr{ALL+} Mirror Subset: Context--Blind AM Sample
AM=blind-mirror-ALL
N = 1,680,633
max digits = 7
Writing "\cmbr{ALL+} Mirror Subset: Context--Blind AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=blind-mirror-ALL.tex
\cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample
AM=blind-mirror-NEQ
N = 583,470
max digits = 6
Writing "\cmbr{NEQ+} Mirror Subset: Context--Blind AM Sample" Table to
  /share/compling/projects/arh2

## Associations with Polarity

### Adverb Polarity

In [271]:
def _pull_polar_samples(all_amdf, neq_amdf, k_lex=2):
    #> l2 strings
    neq_lex = neq_amdf.index.str.split('~').str[1] 
    #> l2 strings that have both + and - rows
    appears_in_both = neq_lex.value_counts().loc[lambda x: x > 1] 
    #> tuple key for sampled lexical items
    sample_lex = tuple(f'~{a}' for a in appears_in_both.to_frame().sample(k_lex).index) 
    
    sample_NEQ = neq_amdf.loc[neq_amdf.index.str.endswith(sample_lex),:].T.convert_dtypes()
    sample_ALL = all_amdf.loc[all_amdf.index.str.endswith(sample_lex),:].T.convert_dtypes()
    if VERBOSE:
        an.nb_display(sample_NEQ.style.background_gradient(axis=1))
        an.nb_display(sample_ALL.style.background_gradient(axis=1))
    else:
        an.nb_display(sample_NEQ.columns.to_series().to_frame('sample'))
    return {'ALL': sample_ALL, 'NEQ':sample_NEQ}

In [274]:
all_super_adv = _load_amdf(dataset='direct', sampling='ALL', unit='adv', polar=True, f_floor=POLAR_ADV_MIN).sort_values(['f2', 'f1'])
neq_super_adv = _load_amdf(dataset='direct', sampling='NEQ', unit='adv', polar=True, f_floor=POLAR_ADV_MIN).sort_values(['f2', 'f1'])

In [307]:
adv_super_samples = _pull_polar_samples(all_amdf=all_super_adv, neq_amdf=neq_super_adv)

key,COM~legally,NEG~legally,COM~easily,NEG~easily
f,"1,339.00","3,213.00","5,357.00","7,638.00"
f1,"3,173,681.00","3,173,681.00","3,173,681.00","3,173,681.00"
f2,"4,552.00","4,552.00","12,995.00","12,995.00"
N,"6,347,362.00","6,347,362.00","6,347,362.00","6,347,362.00"
expF,"2,276.00","2,276.00","6,497.50","6,497.50"
unexpF,-937.00,937.00,"-1,140.50","1,140.50"
unexpR,-0.70,0.29,-0.21,0.15
P1,0.29,0.71,0.41,0.59
P2,0.00,0.00,0.00,0.00
dP1,-0.21,0.21,-0.09,0.09


key,NEG~legally,COM~legally,NEG~easily,COM~easily
f,"3,213.00","28,913.00","7,638.00","114,031.00"
f1,"3,173,681.00","68,787,692.00","3,173,681.00","68,787,692.00"
f2,"32,126.00","32,126.00","121,669.00","121,669.00"
N,"71,961,373.00","71,961,373.00","71,961,373.00","71,961,373.00"
expF,"1,416.84","30,709.16","5,365.91","116,303.09"
unexpF,"1,796.16","-1,796.16","2,272.09","-2,272.09"
unexpR,0.56,-0.06,0.30,-0.02
P1,0.10,0.90,0.06,0.94
P2,0.00,0.00,0.00,0.00
dP1,0.06,-0.06,0.02,-0.02


In [317]:
all_mirror_adv = _load_amdf(dataset='mirror', sampling='ALL', unit='adv', polar=True, f_floor=POLAR_ADV_MIN).sort_values(['f2', 'f1'])
neq_mirror_adv = _load_amdf(dataset='mirror', sampling='NEQ', unit='adv', polar=True, f_floor=POLAR_ADV_MIN).sort_values(['f2', 'f1'])

In [340]:
adv_mirror_samples = _pull_polar_samples(all_amdf=all_mirror_adv, neq_amdf=neq_mirror_adv)

key,POS~entirely,NEG~entirely,NEG~even,POS~even
f,"1,596.00","2,415.00","2,838.00","12,140.00"
f1,"291,735.00","291,735.00","291,735.00","291,735.00"
f2,"4,011.00","4,011.00","14,978.00","14,978.00"
N,"583,470.00","583,470.00","583,470.00","583,470.00"
expF,"2,005.50","2,005.50","7,489.00","7,489.00"
unexpF,-409.50,409.50,"-4,651.00","4,651.00"
unexpR,-0.26,0.17,-1.64,0.38
P1,0.40,0.60,0.19,0.81
P2,0.01,0.01,0.01,0.04
dP1,-0.10,0.10,-0.32,0.32


key,NEG~entirely,POS~entirely,NEG~even,POS~even
f,"2,415.00","7,863.00","2,838.00","58,341.00"
f1,"291,735.00","1,388,898.00","291,735.00","1,388,898.00"
f2,"10,278.00","10,278.00","61,179.00","61,179.00"
N,"1,680,633.00","1,680,633.00","1,680,633.00","1,680,633.00"
expF,"1,784.12","8,493.88","10,619.84","50,559.16"
unexpF,630.88,-630.88,"-7,781.84","7,781.84"
unexpR,0.26,-0.08,-2.74,0.13
P1,0.23,0.77,0.05,0.95
P2,0.01,0.01,0.01,0.04
dP1,0.06,-0.06,-0.13,0.13


#### 💾 Save <font color=DeepPink>Adverb Polarity</font> Samples as $\LaTeX$ tables


In [506]:
# > Save Samples as $\LaTeX$ tables
for sam, sdf in adv_super_samples.items():
    # > Save Samples as $\LaTeX$ tables

    latexify_sample(sdf, cmap=SUP_POLAR_COLOR, neq=(sam=='NEQ'), f_floor=POLAR_ADV_MIN, 
                        comparison_label=get_polar_label('adv')
                        # dry=True, 
                        )

for sam, sdf in adv_mirror_samples.items():
    # > Save Samples as $\LaTeX$ tables

    latexify_sample(sdf, cmap=MIR_POLAR_COLOR, neq=(sam=='NEQ'), f_floor=POLAR_ADV_MIN, 
                        comparison_label=get_polar_label('adv'), 
                        mirror=True
                        )

\cmbr{ALL+} Superset: Adverb Polarity AM Sample
AM=adverb-polarity-super-ALL
N = 71,961,373
max digits = 8
Writing "\cmbr{ALL+} Superset: Adverb Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=adverb-polarity-super-ALL.tex
\cmbr{NEQ+} Superset: Adverb Polarity AM Sample
AM=adverb-polarity-super-NEQ
N = 6,347,362
max digits = 7
Writing "\cmbr{NEQ+} Superset: Adverb Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=adverb-polarity-super-NEQ.tex
\cmbr{ALL+} Mirror Subset: Adverb Polarity AM Sample
AM=adverb-polarity-mirror-ALL
N = 1,680,633
max digits = 7
Writing "\cmbr{ALL+} Mirror Subset: Adverb Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=adverb-polarity-mirror-ALL.tex
\cmbr{NEQ+} Mirror Subset: Adverb Polarity AM Sample
AM=adverb-polarity-mirror-NEQ
N = 583,470
max digits = 6
Writing "\cmbr{NEQ+} Mirr

### Adjective Polarity

In [314]:
all_super_adj = _load_amdf(dataset='direct', sampling='ALL', unit='adj', polar=True, f_floor=POLAR_ADJ_MIN).sort_values(['f2', 'f1'])
neq_super_adj = _load_amdf(dataset='direct', sampling='NEQ', unit='adj', polar=True, f_floor=POLAR_ADJ_MIN).sort_values(['f2', 'f1'])

In [315]:
adj_super_samples = _pull_polar_samples(all_amdf=all_super_adj, neq_amdf=neq_super_adj)

key,NEG~toxic,COM~toxic,NEG~gorgeous,COM~gorgeous
f,675.00,"1,314.00",700.00,"1,500.00"
f1,"3,173,681.00","3,173,681.00","3,173,681.00","3,173,681.00"
f2,"1,989.00","1,989.00","2,200.00","2,200.00"
N,"6,347,362.00","6,347,362.00","6,347,362.00","6,347,362.00"
expF,994.50,994.50,"1,100.00","1,100.00"
unexpF,-319.50,319.50,-400.00,400.00
unexpR,-0.47,0.24,-0.57,0.27
P1,0.34,0.66,0.32,0.68
P2,0.00,0.00,0.00,0.00
dP1,-0.16,0.16,-0.18,0.18


key,NEG~toxic,COM~toxic,NEG~gorgeous,COM~gorgeous
f,675.00,"28,448.00",700.00,"32,156.00"
f1,"3,173,681.00","68,787,692.00","3,173,681.00","68,787,692.00"
f2,"29,123.00","29,123.00","32,856.00","32,856.00"
N,"71,961,373.00","71,961,373.00","71,961,373.00","71,961,373.00"
expF,"1,284.40","27,838.60","1,449.03","31,406.97"
unexpF,-609.40,609.40,-749.03,749.03
unexpR,-0.90,0.02,-1.07,0.02
P1,0.02,0.98,0.02,0.98
P2,0.00,0.00,0.00,0.00
dP1,-0.02,0.02,-0.02,0.02


In [342]:
all_mirror_adj = _load_amdf(dataset='mirror', sampling='ALL', unit='adj', polar=True, f_floor=POLAR_ADJ_MIN).sort_values(['f2', 'f1'])
neq_mirror_adj = _load_amdf(dataset='mirror', sampling='NEQ', unit='adj', polar=True, f_floor=POLAR_ADJ_MIN).sort_values(['f2', 'f1'])

In [343]:
adj_mirror_samples = _pull_polar_samples(all_amdf=all_mirror_adj, neq_amdf=neq_mirror_adj)

key,POS~scared,NEG~scared,POS~free,NEG~free
f,354.00,396.00,799.00,889.00
f1,"291,735.00","291,735.00","291,735.00","291,735.00"
f2,750.00,750.00,"1,688.00","1,688.00"
N,"583,470.00","583,470.00","583,470.00","583,470.00"
expF,375.00,375.00,844.00,844.00
unexpF,-21.00,21.00,-45.00,45.00
unexpR,-0.06,0.05,-0.06,0.05
P1,0.47,0.53,0.47,0.53
P2,0.00,0.00,0.00,0.00
dP1,-0.03,0.03,-0.03,0.03


key,NEG~scared,POS~scared,NEG~free,POS~free
f,396.00,"1,681.00",889.00,"3,896.00"
f1,"291,735.00","1,388,898.00","291,735.00","1,388,898.00"
f2,"2,077.00","2,077.00","4,785.00","4,785.00"
N,"1,680,633.00","1,680,633.00","1,680,633.00","1,680,633.00"
expF,360.54,"1,716.46",830.61,"3,954.39"
unexpF,35.46,-35.46,58.39,-58.39
unexpR,0.09,-0.02,0.07,-0.01
P1,0.19,0.81,0.19,0.81
P2,0.00,0.00,0.00,0.00
dP1,0.02,-0.02,0.01,-0.01


#### 💾 Save <font color=DarkViolet>Adjective Polarity</font> Samples as $\LaTeX$ tables

In [507]:

# > Save Samples as $\LaTeX$ tables
for sam, sdf in adj_super_samples.items():

    latexify_sample(sdf, cmap=SUP_POLAR_COLOR, neq=(sam=='NEQ'), f_floor=POLAR_ADJ_MIN, 
                        comparison_label=get_polar_label('adj')
                        # dry=True, 
                        )

for sam, sdf in adj_mirror_samples.items():
    # > Save Samples as $\LaTeX$ tables

    latexify_sample(sdf, cmap=MIR_POLAR_COLOR, neq=(sam=='NEQ'), f_floor=POLAR_ADJ_MIN, 
                        comparison_label=get_polar_label('adj'), 
                        mirror=True
                        # dry=True, 
                        )

\cmbr{ALL+} Superset: Adjective Polarity AM Sample
AM=adjective-polarity-super-ALL
N = 71,961,373
max digits = 8
Writing "\cmbr{ALL+} Superset: Adjective Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=adjective-polarity-super-ALL.tex
\cmbr{NEQ+} Superset: Adjective Polarity AM Sample
AM=adjective-polarity-super-NEQ
N = 6,347,362
max digits = 7
Writing "\cmbr{NEQ+} Superset: Adjective Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=adjective-polarity-super-NEQ.tex
\cmbr{ALL+} Mirror Subset: Adjective Polarity AM Sample
AM=adjective-polarity-mirror-ALL
N = 1,680,633
max digits = 7
Writing "\cmbr{ALL+} Mirror Subset: Adjective Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=adjective-polarity-mirror-ALL.tex
\cmbr{NEQ+} Mirror Subset: Adjective Polarity AM Sample
AM=adjective-polarity-mirror-NEQ
N = 583,47

### Bigram Polarity

In [345]:
all_mirror_bigram = _load_amdf(dataset='mirror', sampling='ALL', unit='bigram', polar=True, f_floor=POLAR_BIG_MIN).sort_values(['f2', 'f1'])
neq_mirror_bigram = _load_amdf(dataset='mirror', sampling='NEQ', unit='bigram', polar=True, f_floor=POLAR_BIG_MIN).sort_values(['f2', 'f1'])
all_super_bigram = _load_amdf(dataset='direct', sampling='ALL', unit='bigram', polar=True, f_floor=POLAR_BIG_MIN).sort_values(['f2', 'f1'])
neq_super_bigram = _load_amdf(dataset='direct', sampling='NEQ', unit='bigram', polar=True, f_floor=POLAR_BIG_MIN).sort_values(['f2', 'f1'])

In [365]:
bigram_mirror_samples = _pull_polar_samples(all_amdf=all_mirror_bigram, neq_amdf=neq_mirror_bigram)

key,POS~so_great,NEG~so_great,POS~too_big,NEG~too_big
f,213.00,280.00,377.00,787.00
f1,"291,735.00","291,735.00","291,735.00","291,735.00"
f2,493.00,493.00,"1,164.00","1,164.00"
N,"583,470.00","583,470.00","583,470.00","583,470.00"
expF,246.50,246.50,582.00,582.00
unexpF,-33.50,33.50,-205.00,205.00
unexpR,-0.16,0.12,-0.54,0.26
P1,0.43,0.57,0.32,0.68
P2,0.00,0.00,0.00,0.00
dP1,-0.07,0.07,-0.18,0.18


key,NEG~so_great,POS~so_great,NEG~too_big,POS~too_big
f,280.00,979.00,787.00,"1,754.00"
f1,"291,735.00","1,388,898.00","291,735.00","1,388,898.00"
f2,"1,259.00","1,259.00","2,541.00","2,541.00"
N,"1,680,633.00","1,680,633.00","1,680,633.00","1,680,633.00"
expF,218.55,"1,040.45",441.08,"2,099.92"
unexpF,61.45,-61.45,345.92,-345.92
unexpR,0.22,-0.06,0.44,-0.20
P1,0.22,0.78,0.31,0.69
P2,0.00,0.00,0.00,0.00
dP1,0.05,-0.05,0.14,-0.14


In [383]:
bigram_super_samples = _pull_polar_samples(all_amdf=all_super_bigram, neq_amdf=neq_super_bigram)

key,COM~very_deep,NEG~very_deep,COM~too_bad,NEG~too_bad
f,706.00,896.00,"2,499.00","19,488.00"
f1,"3,173,681.00","3,173,681.00","3,173,681.00","3,173,681.00"
f2,"1,602.00","1,602.00","21,987.00","21,987.00"
N,"6,347,362.00","6,347,362.00","6,347,362.00","6,347,362.00"
expF,801.00,801.00,"10,993.50","10,993.50"
unexpF,-95.00,95.00,"-8,494.50","8,494.50"
unexpR,-0.13,0.11,-3.40,0.44
P1,0.44,0.56,0.11,0.89
P2,0.00,0.00,0.00,0.01
dP1,-0.06,0.06,-0.39,0.39


key,NEG~very_deep,COM~very_deep,NEG~too_bad,COM~too_bad
f,896.00,"15,241.00","19,488.00","53,191.00"
f1,"3,173,681.00","68,787,692.00","3,173,681.00","68,787,692.00"
f2,"16,137.00","16,137.00","72,679.00","72,679.00"
N,"71,961,373.00","71,961,373.00","71,961,373.00","71,961,373.00"
expF,711.68,"15,425.32","3,205.33","69,473.67"
unexpF,184.32,-184.32,"16,282.67","-16,282.67"
unexpR,0.21,-0.01,0.84,-0.31
P1,0.06,0.94,0.27,0.73
P2,0.00,0.00,0.01,0.00
dP1,0.01,-0.01,0.22,-0.22


#### 💾 Save <font color=DodgerBlue>Bigram Polarity</font> Samples as $\LaTeX$ tables


In [508]:
# > Save Samples as $\LaTeX$ tables
# > mirror
for sam, sdf in adj_mirror_samples.items():

    latexify_sample(sdf, cmap=MIR_POLAR_COLOR, 
                    neq=(sam=='NEQ'), f_floor=POLAR_BIG_MIN, 
                        comparison_label=get_polar_label('bigram'), 
                        mirror=True)

# > super
for sam, sdf in adj_super_samples.items():

    latexify_sample(sdf, cmap=SUP_POLAR_COLOR, neq=(sam=='NEQ'), f_floor=POLAR_BIG_MIN, 
                        comparison_label=get_polar_label('bigram'), 
                        mirror=False)

\cmbr{ALL+} Mirror Subset: Bigram Polarity AM Sample
AM=bigram-polarity-mirror-ALL
N = 1,680,633
max digits = 7
Writing "\cmbr{ALL+} Mirror Subset: Bigram Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=bigram-polarity-mirror-ALL.tex
\cmbr{NEQ+} Mirror Subset: Bigram Polarity AM Sample
AM=bigram-polarity-mirror-NEQ
N = 583,470
max digits = 6
Writing "\cmbr{NEQ+} Mirror Subset: Bigram Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=bigram-polarity-mirror-NEQ.tex
\cmbr{ALL+} Superset: Bigram Polarity AM Sample
AM=bigram-polarity-super-ALL
N = 71,961,373
max digits = 8
Writing "\cmbr{ALL+} Superset: Bigram Polarity AM Sample" Table to
  /share/compling/projects/arh234/OverleafDissertex/assets/tables/ch4/samples/AM=bigram-polarity-super-ALL.tex
\cmbr{NEQ+} Superset: Bigram Polarity AM Sample
AM=bigram-polarity-super-NEQ
N = 6,347,362
max digits = 7
Writing "\cmbr{NEQ+